### DB INSERT_yearly_album_chart

In [1]:
import pandas as pd
import pymysql

In [2]:
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'stock_pj'
                , charset='utf8'
                ) 

In [3]:
cur = conn.cursor()
cur.execute('SHOW TABLES') # 연동 확인
cur.fetchall() # 테이블 이름 확인

(('album',), ('singer_agency',), ('stock',), ('yearly_album_chart',))

In [4]:
yearly_album_chart_df = pd.read_csv('data/yearly_album_chart_df.csv')
yearly_album_chart_df.head()

,year,rank,title,singer,sales
0,2018,1,LOVE YOURSELF 結 ''Answer'',방탄소년단,2197808
1,2018,2,LOVE YOURSELF 轉 ''Tear'',방탄소년단,1849537
2,2018,3,DON''T MESS UP MY TEMPO - The 5th Album,EXO,1452030
3,2018,4,0+1=1 (I PROMISE YOU),Wanna One (워너원),782562
4,2018,5,1÷χ=1 (UNDIVIDED),Wanna One (워너원),641131


- DB에 'singer_agency' 정보를 가져와 3사 앨범 순위 정보만 추출하기

In [5]:
cur.execute('SELECT * FROM singer_agency;')
singer_agency =  cur.fetchall()
singer_agency_df = pd.DataFrame(singer_agency)
singer_agency_df.columns = ['singer', 'agency_name']
singer_agency_df.head()

,singer,agency_name
0,2PM,JYP
1,DAY6,JYP
2,ITZY (있지),JYP
3,NMIXX,JYP
4,Stray Kids (스트레이 키즈),JYP


In [7]:
total_yearly_album_df = pd.DataFrame()
for singer_1 in yearly_album_chart_df['singer']:
    for singer_2 in singer_agency_df['singer']:
        if singer_1 == singer_2:
            tmp_df = yearly_album_chart_df[yearly_album_chart_df['singer'] == singer_1]
            total_yearly_album_df = pd.concat([total_yearly_album_df, tmp_df])

In [8]:
total_yearly_album_df.drop_duplicates(inplace=True)

In [9]:
total_yearly_album_df.shape

(127, 5)

In [16]:
total_yearly_album_df.sort_values(['year', 'rank'], inplace=True)

In [20]:
total_yearly_album_df.reset_index(drop=True, inplace=True)

In [24]:
total_yearly_album_df[total_yearly_album_df['year'] == 2018]

,year,rank,title,singer,sales
0,2018,3,DON''T MESS UP MY TEMPO - The 5th Album,EXO,1452030
1,2018,7,LOVE SHOT - The 5th Album Repackage,EXO,499849
2,2018,10,What is Love?,TWICE,348797
3,2018,13,NCT 2018 EMPATHY,NCT,330335
4,2018,15,YES or YES,TWICE,325761
5,2018,16,Summer Nights,TWICE,314421
6,2018,17,NCT #127 Regular-Irregular - The 1st Album,NCT 127,292341
7,2018,21,SQUARE UP,BLACKPINK,230656
8,2018,22,We Go Up - The 2nd Mini Album,NCT DREAM,202626
9,2018,23,The year of ''YES'',TWICE,196034


--------------

In [12]:
# insert stock table
for i in range(len(total_yearly_album_df)):
    cur.execute(f"INSERT INTO yearly_album_chart (year, ranking, singer, title, sales) VALUES \
                ({total_yearly_album_df['year'][i]}, {total_yearly_album_df['rank'][i]}, '{total_yearly_album_df['singer'][i]}', '{total_yearly_album_df['title'][i]}', {total_yearly_album_df['sales'][i]});")

In [13]:
# db 반영
conn.commit()

In [14]:
conn.close()